In [1]:
import cmipld
from cmipld.utils.offline import LD_server, replace_loader

In [ ]:
# Set up offline override for the current repository

ldpath = cmipld.utils.git.ldpath()
localserver = LD_server(copy = [[ldpath,'current_repo']])
localhost = localserver.start_server(8082)
repo_url = cmipld.utils.git.url()
io_url = cmipld.utils.git.url2io(repo_url)
replace_loader(localhost,[(io_url,'current_repo')] )




Copying the repo into LocalServer  /Users/daniel.ellis/WIPwork/CMIP6Plus_CVs/src-data/ --> current_repo
Repositories copied into <TemporaryDirectory '/var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_qlux04pi2025-01-12T23:09:10'>
Created SSL certificates in: /var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_qlux04pi2025-01-12T23:09:10
Serving /var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_qlux04pi2025-01-12T23:09:10 at https://localhost:8082
make test for url2io
Setting up location forwarding for:
 -  https://wcrp-cmip.github.io/CMIP6Plus_CVs/ >>> https://localhost:8082/current_repo/



.+...+.............+......+.....+......+...+.+...+..+.........+++++++++++++++++++++++++++++++++++++++++++++*.......+...............+......+..+.........+....+.........+..+....+..+.......+.....+.+.........+..+.+..+.+...........+.............+...+.....+...+......+.+.....+..........+.........+.....+...+..........+.....+.+...........+....+.....+......+++++++++++++++++++++++++++++++++++++++++++++*....+......+.........................+............+...+..+....+...+..+.............+...+...+............+++++
...+.+........+.+..+...+......+....+...............+..+..........+..+...+.........+.+............+.....+.......+...+........+............+......+.......+..+......+.+...............+.....+.............+..+.............+..+.+...+...........+....+..+...+.......+..+.+......+.....+...............+....+......+++++++++++++++++++++++++++++++++++++++++++++*.+...+........+++++++++++++++++++++++++++++++++++++++++++++*.+.+.....+.........+...+................+.....+....+..+..................+...+.......+

In [ ]:
# This means we can make 'locally hosted' content appear as an external resource in our PYLD instance

testfile = f'{io_url}project/'

from pyld import jsonld
jsonld.expand(testfile)

In [ ]:
# or the cmipld library loaders.
cmipld.get(testfile)


In [ ]:
item = 'tables'

cmipld.get(f'{io_url}project/{item}.json')

[{'@context': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/_context_',
  'id': 'cmip6plus:project/tables',
  'type': ['wcrp:project', 'cmip6plus', 'wcrp:tables'],
  'tables': [{'id': 'variables:tables/acmon',
    'type': ['wcrp:tables', 'variables'],
    'Conventions': 'CF-1.7 CMIP-6.5',
    'approx_interval': 30.0,
    'checksum': '',
    'cmor_version': '3.8.0',
    'data_specs_version': '6.5.0.0',
    'generic_levels': '',
    'int_missing_value': '-999',
    'missing_value': '1e20',
    'product': {'id': 'universal:product/model-output',
     'type': ['wcrp:product', 'universal'],
     'description': 'Output produced from a model simulation. This includes the various data points and metrics generated as a result of running climate models, which simulate physical, chemical, and biological processes affecting the climate system.',
     'kind': 'model-output'},
    'table_date': '2023-11-16',
    'table_id': 'ACmon'},
   {'id': 'variables:tables/acmonz',
    'type': ['wcrp:table

In [46]:
from pyld import jsonld
jsonld.frame(cmipld.get(f'{io_url}project/{item}.json'), {
    '@context': [
        f'{io_url}project/{item}.json', 
        ],
    "tables":{
            "table_id":'',
            "@explicit": True,
            }
    })

{'@context': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/tables.json',
 'id': 'cmip6plus:project/tables',
 'type': ['wcrp:project', 'cmip6plus', 'wcrp:tables'],
 'tables': [{'id': 'variables:tables/acmon',
   'type': ['wcrp:tables', 'variables'],
   'table_id': 'ACmon'},
  {'id': 'variables:tables/acmonz',
   'type': ['wcrp:tables', 'variables'],
   'table_id': 'ACmonZ'},
  {'id': 'variables:tables/ae1hr',
   'type': ['wcrp:tables', 'variables'],
   'table_id': 'AE1hr'},
  {'id': 'variables:tables/ae3hrpt',
   'type': ['wcrp:tables', 'variables'],
   'table_id': 'AE3hrPt'},
  {'id': 'variables:tables/ae3hrptlev',
   'type': ['wcrp:tables', 'variables'],
   'table_id': 'AE3hrPtLev'},
  {'id': 'variables:tables/ae6hr',
   'type': ['wcrp:tables', 'variables'],
   'table_id': 'AE6hr'},
  {'id': 'variables:tables/ae6hrpt',
   'type': ['wcrp:tables', 'variables'],
   'table_id': 'AE6hrPt'},
  {'id': 'variables:tables/ae6hrptlev',
   'type': ['wcrp:tables', 'variables'],
   'table_id':

In [ ]:
jsonld.frame(f'{cmipld.mapping["variables"]}tables/apsubhrptsite',{"@context":[f'{cmipld.mapping["variables"]}tables/apsubhrptsite',{}],"table_id":{},"@explicit":True})

{'@context': 'https://wcrp-cmip.github.io/MIP-variables/tables/apsubhrptsite',
 'id': 'variables:tables/apsubhrptsite',
 'type': ['wcrp:tables', 'variables'],
 'table_id': 'APsubhrPtSite'}